In [43]:
#imports for Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

from numpy.random import rand, randn
from numpy import hstack, zeros, ones
from matplotlib import pyplot

In [44]:
def generate_real_samples(n):
  X1 = rand(n) - 0.5
  X2 = X1 * X1
  #reshape
  X1 = X1.reshape(n,1)
  X2 = X2.reshape(n,1)
  X = hstack((X1, X2))
  #generate class labels
  y = ones((n,1))
  return X, y

In [45]:
#define discriminator model
def define_discriminator(n_inputs = 2):
  model = Sequential()
  model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim = n_inputs))
  model.add(Dense(1, activation='sigmoid'))
  #compile model
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  return model

In [46]:
def define_generator(latent_dim, n_outputs = 2):
  model = Sequential()
  model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim = latent_dim))
  model.add(Dense(n_outputs, activation='linear'))
  return model

In [47]:
def define_gan(generator, discriminator):
  discriminator.trainable = False
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

In [48]:
def generate_latent_points(latent_dim, n):
  x_input = randn(n * latent_dim)
  x_input = x_input.reshape(n, latent_dim)
  return x_input

In [49]:
def generate_fake_samples(generator, latent_dim, n):
  x_input = generate_latent_points(latent_dim, n)
  #predict output
  X = generator.predict(x_input)
  #plot result
  y = zeros((n,1))
  return X, y


In [50]:
def train(g_model, d_model, gan_model, latent_dim, n_epochs = 10000, n_batch = 128, n_eval = 2000):
  half_batch = int(n_batch/2)
  for i in range(n_epochs):
    # generate real/fake points
    x_real, y_real = generate_real_samples(half_batch)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    # train discriminator
    d_model.train_on_batch(x_real, y_real)
    d_model.train_on_batch(x_fake, y_fake)
    # generate latent points
    x_gan = generate_latent_points(latent_dim, n_batch)
    y_gan = ones((n_batch,1))
    # train gan/generator
    gan_model.train_on_batch(x_gan, y_gan)
    # evaluate
    if(i + 1) % n_eval == 0:
      summarize_performance(i, g_model, d_model, latent_dim)



In [51]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n = 100):
  x_real, y_real = generate_real_samples(n)
  _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)

  x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
  _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
  
  print(epoch, acc_real, acc_fake)

  pyplot.scatter(x_real[:, 0], x_real[:, 1], color = 'red')
  pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color = 'blue')

  filename = 'generated_plot_e%03d.png' % (epoch + 1)
  pyplot.savefig(filename)
  pyplot.close()

In [53]:
#define latent space, gan, and plot results
latent_dim = 5
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)

#train model
train(generator, discriminator, gan_model, latent_dim)

1999 0.5099999904632568 0.6499999761581421
3999 0.6800000071525574 0.5699999928474426
5999 0.550000011920929 0.550000011920929
7999 0.5400000214576721 0.6399999856948853
9999 0.6700000166893005 0.5099999904632568


In [54]:
discriminator.save('discriminator.h5')
generator.save('generator.h5')
gan_model.save('1D_gan.h5')

In [67]:
x_fake, _ = generate_fake_samples(generator, 5, 10)
print(x_fake)

[[-0.29692265  0.09821919]
 [ 0.27843598  0.08123943]
 [-0.24144065  0.06306869]
 [ 0.00087422  0.00925259]
 [-0.0021473   0.00428645]
 [-0.22855781  0.05171187]
 [ 0.36621433  0.14318857]
 [ 0.18474434  0.05218518]
 [ 0.29127324  0.08804817]
 [-0.2563214   0.07167468]]
